In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24414
2,Huelva,Confirmados PDIA 14 días,728
3,Huelva,Tasa PDIA 14 días,"141,8633201473196"
4,Huelva,Confirmados PDIA 7 días,242
5,Huelva,Total Confirmados,24642
6,Huelva,Curados,14718
7,Huelva,Fallecidos,280


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24414.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6518.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2820 personas en los últimos 7 días 

Un positivo PCR cada 682 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24414.0,242.0,728.0,513170.0,47.157862,141.863320,29.0
Huelva-Costa,14888.0,133.0,445.0,289548.0,45.933662,153.687817,15.0
Condado-Campiña,6796.0,97.0,242.0,156231.0,62.087550,154.898836,15.0
Huelva (capital),6518.0,51.0,211.0,143837.0,35.456802,146.693827,8.0
Lepe,1982.0,24.0,63.0,27880.0,86.083214,225.968436,4.0
Bollullos Par del Condado,602.0,18.0,42.0,14387.0,125.112949,291.930215,4.0
Almonte,838.0,17.0,62.0,24507.0,69.367936,252.988942,3.0
Villalba del Alcor,442.0,4.0,6.0,3366.0,118.835413,178.253119,3.0
Moguer,816.0,20.0,37.0,21867.0,91.462020,169.204738,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,138.0,3.0,9.0,2287.0,131.176213,393.528640,0.0
Villanueva de los Castillejos,149.0,5.0,10.0,2825.0,176.991150,353.982301,0.0
Villablanca,172.0,7.0,9.0,2885.0,242.634315,311.958406,1.0
Cartaya,923.0,18.0,62.0,20083.0,89.628044,308.718817,1.0
Bollullos Par del Condado,602.0,18.0,42.0,14387.0,125.112949,291.930215,4.0
Rociana del Condado,308.0,8.0,21.0,7939.0,100.768359,264.516942,0.0
Almonte,838.0,17.0,62.0,24507.0,69.367936,252.988942,3.0
Lepe,1982.0,24.0,63.0,27880.0,86.083214,225.968436,4.0
Villalba del Alcor,442.0,4.0,6.0,3366.0,118.835413,178.253119,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Isla Cristina,1975.0,5.0,29.0,21393.0,23.372131,135.558360,0.0,0.172414
Valverde del Camino,415.0,1.0,5.0,12750.0,7.843137,39.215686,1.0,0.200000
Gibraleón,508.0,3.0,13.0,12737.0,23.553427,102.064850,0.0,0.230769
Huelva (capital),6518.0,51.0,211.0,143837.0,35.456802,146.693827,8.0,0.241706
Sierra de Huelva-Andévalo Central,2302.0,6.0,23.0,67391.0,8.903266,34.129186,1.0,0.260870
Aljaraque,918.0,4.0,15.0,21474.0,18.627177,69.851914,2.0,0.266667
Almonte,838.0,17.0,62.0,24507.0,69.367936,252.988942,3.0,0.274194
Cartaya,923.0,18.0,62.0,20083.0,89.628044,308.718817,1.0,0.290323
Huelva-Costa,14888.0,133.0,445.0,289548.0,45.933662,153.687817,15.0,0.298876
